# Wires från dgn till gdb
## fas och topp för sig
## littera som attribut

In [1]:
# Läs settings
import json

settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEKNNO_postgis.json"
#settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEVPLI.json"
#settings_file = r"C:\Users\SE1K4H\Desktop\SVK-Analys-Filer\settings_test_PostGIS.json" # WOrking on Elsas's computer
#settings_file = r"C:\SVK_utveckling\settings_SEVPLI.json"

with open(settings_file, 'r', encoding='utf-8') as file:
    settings = json.load(file)

#print(settings)
print(json.dumps(settings, indent=4))
print("körning av cell klar")

{
    "run_ID": "250519",
    "local_folder": "C:/Uppdrag/2024/SVK_2024/pythonk\u00f6rningar",
    "results_gdb_template": "Q:/Projekt/Data_2024/results_template.gdb",
    "working_gdb_template": "Q:/Projekt/Data_2024/working_template.gdb",
    "wires_gdb_template": "Q:/Projekt/Data_2024/wires_template.gdb",
    "domains_folder": "Q:/Projekt/Data_2024/Underlag_SVK/Domains",
    "powerline_list": "Q:/Projekt/Data_2024/styrfiler/veg_kanttrad_250519.txt",
    "scandate_file": "Q:/Projekt/Data_2024/skanningsdatum/skanningsdatum.txt",
    "powerlines_folder": "Q:/Projekt/Analys_2024/ledningar",
    "dgn_folder": "Q:/Projekt/Analys_2024/DGN_2024",
    "order": "Q:/Projekt/Data_2024/Underlag_SVK/Bestallningsunderlag_2024.gdb",
    "LG_polygons": "Q:/Projekt/Data_2024/urval_bestallningsunderlag_2024.gdb/GNG_LEDNINGSGATA",
    "station_polygons": "Q:/Projekt/Data_2024/urval_bestallningsunderlag_2024.gdb/GNG_STATIONSOMRADE",
    "ogr2ogr_path": "C:/Program Files/QGIS 3.40.4/bin/ogr2ogr.exe",
   

In [ ]:
import os, glob
# import arcpy
import pandas as pd
import shutil

run_ID = settings["run_ID"]
powerline_list = settings["powerline_list"]
local_dir = settings["local_folder"]
dgn_dir = settings["dgn_folder"]
wires_gdb_template = settings["wires_gdb_template"]
domains_folder = settings["domains_folder"]

cvd_LEDNINGSGATA_path = os.path.join(domains_folder, "cvd_LEDNINGSGATA.txt")
wires_gdb = os.path.join(local_dir, f"wires_{run_ID}.gdb")

# Create wire gdb as copy of template
if os.path.exists(wires_gdb):
    print(f"{wires_gdb} finns redan")
else: 
    shutil.copytree(wires_gdb_template, wires_gdb)
arcpy.env.overwriteOutput = True
def wires_to_gdb(row):
    # Ange i definitionen att row är en pandas serie
    LG = row["LG"]
    line = row["line"]
    littera = row["Littera"]
    LG_code = cvd_LEDNINGSGATA[LG]
  
    print(f"Påbörjar export av fas- och topplinor för {LG}_{line}, {littera}")
  
    dgn = os.path.join(dgn_dir, f"{LG}_{line}.dgn")
    arcpy.env.workspace = dgn_dir # Varför ska den ha dgn dir som workspace?
  
    # faslinor
    fc_fas = f"{LG}_{line}_fas"
    # Create a value table that will hold the input feature classes for Merge
    vTab_fas = arcpy.ValueTable()
    layer_fas = "Layer_fas"
    arcpy.Delete_management(layer_fas)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_fas, "\"Level\" = 1")
    vTab_fas.addRow(layer_fas)
      
    # topplinor
    fc_topp = f"{LG}_{line}_topp"
    # Create a value table that will hold the input feature classes for Merge
    vTab_topp = arcpy.ValueTable()
    layer_topp = "Layer_topp"
    arcpy.Delete_management(layer_topp)
    arcpy.MakeFeatureLayer_management(dgn + '/Polyline', layer_topp, "\"Level\" = 2")
    vTab_topp.addRow(layer_topp)
  
    # Merge the CAD features into one feature class
    arcpy.env.workspace = wires_gdb
  
    # faslinor
    arcpy.Merge_management(vTab_fas, fc_fas)
    # Add and populate littera field
    arcpy.management.AddField(fc_fas, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_fas, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_fas, "LG", "LONG")
    arcpy.management.CalculateField(fc_fas, "LG", int(LG_code))
    arcpy.management.AddField(fc_fas, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_fas, "lintyp", "'ledare'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
  
    # ta bort fält som inte ska användas DETTA BEHÖVER GÖRAS PÅ BÅDE FAS OCH TOPP MEN NUVARANDE 
    # UPPREPNING ÄR KLUMPIG
    keep_fields = ["LG", "Littera", "lintyp", "Shape", "Shape_Length", "OBJECTID"]
    all_fields = arcpy.ListFields(fc_fas)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(fc_fas, fields_to_delete)   
    # Räkna faslinor, varna om inga finns
    arcpy.management.DefineProjection(fc_fas, sr)
    result = arcpy.management.GetCount(fc_fas)
    count = int(result[0])
    if count == 0:
        print(f"VARNING: Inga faslinor hittades i level 1 i {LG}_{line}.dgn")
  
    # topplinor
    arcpy.Merge_management(vTab_topp, fc_topp)
    # Add and populate littera field
    arcpy.management.AddField(fc_topp, "Littera", "TEXT")
    arcpy.management.CalculateField(fc_topp, "Littera", "".join(("'",littera,"'")))
    arcpy.management.AddField(fc_topp, "LG", "LONG")
    arcpy.management.CalculateField(fc_topp, "LG", int(LG_code))
    arcpy.management.AddField(fc_topp, "lintyp", "TEXT")
    arcpy.management.CalculateField(fc_topp, "lintyp", "'topplina'")
    # domains
    arcpy.management.AssignDomainToField(fc_fas, "LG", "cvd_LEDNINGSGATA")
    arcpy.management.AssignDomainToField(fc_fas, "Littera", "cvd_LITTERA_LEDNING")
  
    # ta bort fält som inte ska användas
    keep_fields = ["LG", "Littera", "lintyp", "Shape", "Shape_Length", "OBJECTID"]
    all_fields = arcpy.ListFields(fc_topp)
    fields_to_delete = [field.name for field in all_fields if field.name not in keep_fields]
    if fields_to_delete:
        arcpy.DeleteField_management(fc_topp, fields_to_delete)
  
    arcpy.management.DefineProjection(fc_topp, sr)
    # Räkna topplinor, varna om inga finns
    result = arcpy.management.GetCount(fc_topp)
    count = int(result[0])
    if count == 0:
        print(f"VARNING: Inga topplinor hittades i level 2 i {LG}_{line}.dgn")
      
    # Städa bort tillfälliga lager från minnet
    arcpy.env.workspace = dgn_dir
    arcpy.Delete_management(layer_fas)
    arcpy.Delete_management(layer_topp)
       
# Sätt detta i en main-funktion för tydlighet?
sr = arcpy.SpatialReference("SWEREF99_TM", "RH2000")
powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
df_cvd_LEDNINGSGATA = pd.read_csv(cvd_LEDNINGSGATA_path, sep="\t", header=0)
cvd_LEDNINGSGATA = {df_cvd_LEDNINGSGATA.Description[i]: df_cvd_LEDNINGSGATA.Code[i] for i in range(len(df_cvd_LEDNINGSGATA))}
powerlines_df.apply(wires_to_gdb, axis=1)
print("körning av cell klar")

# Kontroll

In [2]:
arcpy.env.workspace = wires_gdb

feature_classes = arcpy.ListFeatureClasses()
print(f"Antal featureklasser: {len(feature_classes)}")

for fc in feature_classes:
    feature_count = int(arcpy.management.GetCount(fc)[0])
    print(f"{fc}: {feature_count} features")
print("körning av cell klar")

Antal featureklasser: 100
LG030_1_fas: 594 features
LG030_1_topp: 396 features
LG030_2_fas: 597 features
LG030_2_topp: 398 features
LG031_1_fas: 9 features
LG031_1_topp: 6 features
LG031_2_fas: 10 features
LG031_2_topp: 6 features
LG031_3_fas: 207 features
LG031_3_topp: 138 features
LG031_4_fas: 72 features
LG031_4_topp: 48 features
LG031_5_fas: 102 features
LG031_5_topp: 68 features
LG031_6_fas: 1398 features
LG031_6_topp: 930 features
LG043_1_fas: 900 features
LG043_1_topp: 600 features
LG043_2_fas: 357 features
LG043_2_topp: 238 features
LG045_1_fas: 567 features
LG045_1_topp: 378 features
LG045_2_fas: 1473 features
LG045_2_topp: 982 features
LG045_3_fas: 24 features
LG045_3_topp: 11 features
LG045_4_fas: 21 features
LG045_4_topp: 10 features
LG051_1_fas: 811 features
LG051_1_topp: 540 features
LG051_2_fas: 1005 features
LG051_2_topp: 674 features
LG051_3_fas: 184 features
LG051_3_topp: 76 features
LG051_4_fas: 33 features
LG051_4_topp: 22 features
LG051_5_fas: 807 features
LG051_5_

# Convert wires gdb to PostGIS

## Load Settings from File

In [7]:
import json
import os

# settings_file = r"C:\Users\SE1K4H\Desktop\SVK-Analys-Filer\settings_test_PostGIS.json" # Working on Elsa's computer
settings_file = r"Q:\Projekt\Data_2024\styrfiler\settings_SEKNNO_postgis.json" # Working on Karin's computer

try:
    if not os.path.exists(settings_file):
        raise FileNotFoundError(f"Could not find {settings_file}")

    with open(settings_file, 'r', encoding='utf-8') as file:
        settings = json.load(file)

    run_ID = settings["run_ID"]
    powerline_list = settings["powerline_list"]
    local_dir = settings["local_folder"]
    dgn_dir = settings["dgn_folder"]
    wires_gdb_template = settings["wires_gdb_template"]
    domains_folder = settings["domains_folder"]
    powerlines_folder = settings["powerlines_folder"]
    module_path = settings["modules"] 
    ogr2ogr_path = settings["ogr2ogr_path"]   
    proj_lib_path = settings["proj_lib_path"]
    gdal_data_path = settings["gdal_data_path"]
    DEFAULT_DB_NAME = settings["default_db_name"]
    DB_NAME = settings["db_name"]
    USER = settings["db_user"]
    PASSWORD = settings["db_password"]
    HOST = settings["host"]
    PORT = settings["port"] 

    if None in (run_ID, powerline_list, local_dir, dgn_dir, wires_gdb_template, domains_folder, powerlines_folder, module_path, ogr2ogr_path, proj_lib_path, gdal_data_path, DEFAULT_DB_NAME, DB_NAME, USER, PASSWORD, HOST, PORT):
        raise KeyError(f"One or more keys are missing in {settings_file}")
    
    print(f"Settings loaded successfully.")
except json.JSONDecodeError as e:
    print(f"Error: Invalid JSON format in {settings_file}")
except KeyError as e:
    print(f"Error: {e}")
except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Error: Unexpected error: {e}")

Settings loaded successfully.


## Load PostGIS methods

In [18]:
import psycopg2 as psycopg
from psycopg2 import sql#, errors

# TODO can move these kind of methods to utils or similiar?

def connect_to_database(db_name):
    try:
        conn = psycopg.connect(
            dbname=db_name, user=USER, password=PASSWORD, host=HOST, port=PORT
        )
        conn.autocommit = True
        return conn
    except psycopg.OperationalError as e:
        raise RuntimeError(
            f"Kunde inte ansluta till databasen '{db_name}'. Kontrollera att PostgreSQL-tjänsten är startad och att inloggningsuppgifterna är rätt.\n{e}"
        )

def create_database_if_not_exists(conn, db_name):
    try:
        with conn.cursor() as cur:
            cur.execute("SELECT 1 FROM pg_database WHERE datname = %s", (db_name,))
            exists = cur.fetchone()
            if not exists:
                cur.execute(
                    sql.SQL("CREATE DATABASE {} ENCODING 'UTF8' TEMPLATE template0")
                    .format(sql.Identifier(db_name))
                )
                print(f"Databasen '{db_name}' skapades.")
            else:
                print(f"Databasen '{db_name}' finns redan.")
    except Exception as e:
        raise RuntimeError(f"Kunde inte skapa databasen: {e}")

def enable_postgis_extension(conn):
    try:
        with conn.cursor() as cur:
            cur.execute("CREATE EXTENSION IF NOT EXISTS postgis;")
            print("PostGIS har aktiverats.")
    except Exception as e:
        raise RuntimeError(f"Misslyckades med att aktivera PostGIS: {e}")


## Set up PostGIS Database

In [13]:
import psycopg2 as psycopg
from psycopg2 import sql, errors

try:
    print("Ansluter till default-databasen...")
    conn_default = connect_to_database(DEFAULT_DB_NAME)
    create_database_if_not_exists(conn_default, DB_NAME)
    conn_default.close()

    print(f"Ansluter till '{DB_NAME}' för att aktivera PostGIS...")
    conn_target = connect_to_database(DB_NAME)
    enable_postgis_extension(conn_target)
    conn_target.close()

    print(f"Databasen '{DB_NAME}' är klar med PostGIS aktiverat.")

except Exception as e:
    print("\nFEL:")
    print(e)

Ansluter till default-databasen...
Databasen 'leverans_250519' finns redan.
Ansluter till 'leverans_250519' för att aktivera PostGIS...
PostGIS har aktiverats.
Databasen 'leverans_250519' är klar med PostGIS aktiverat.


## Convert wires gdb to PostGIS

In [14]:
import os
import subprocess
import pandas as pd


#TODO in general in all cells, take a look att exception handling
PG_CONNECTION = f"PG:host={HOST} dbname={DB_NAME} user={USER} password={PASSWORD} port={PORT}"

def convert_gdb_to_postGIS(row):
    LG = row["LG"]
    line = row["line"]  
    table_name = f"{LG}_{line}_fas"
    ogr_command = [
        ogr2ogr_path,
        "-f", "PostgreSQL",
        PG_CONNECTION,
        wires_gdb,
        "-a_srs", "EPSG:3006",
        "-nln", table_name,
        "-nlt", "MULTILINESTRINGZ",
        "-dim", "3",
        "-overwrite", #TODO change this when there are several dgn-files, eg. append
        table_name        
    ]

    try:
        os.environ["PROJ_LIB"] = proj_lib_path #TODO potentially solve this in another way. Now it has to be done in order to find -a_srs EPSG:3006
        os.environ["GDAL_DATA"] = gdal_data_path
        subprocess.run(ogr_command, check=True, capture_output=True, text=True)
        print(f"Successfully imported {wires_gdb} into PostGIS as '{table_name}'.") #TODO verify this
    except subprocess.CalledProcessError as e:
        print(f"Error: Could not import {wires_gdb} into PostGIS. {e}")

try:   
    wires_gdb = os.path.join(local_dir, f"wires_{run_ID}.gdb")
    powerlines_df = pd.read_csv(powerline_list, sep="\t", header=0)
    powerlines_df.apply(convert_gdb_to_postGIS, axis=1)
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

Successfully imported C:/Uppdrag/2024/SVK_2024/pythonkörningar\wires_250519.gdb into PostGIS as 'LG030_1_fas'.
Successfully imported C:/Uppdrag/2024/SVK_2024/pythonkörningar\wires_250519.gdb into PostGIS as 'LG030_2_fas'.
Successfully imported C:/Uppdrag/2024/SVK_2024/pythonkörningar\wires_250519.gdb into PostGIS as 'LG031_1_fas'.
Successfully imported C:/Uppdrag/2024/SVK_2024/pythonkörningar\wires_250519.gdb into PostGIS as 'LG031_2_fas'.
Successfully imported C:/Uppdrag/2024/SVK_2024/pythonkörningar\wires_250519.gdb into PostGIS as 'LG031_3_fas'.
Successfully imported C:/Uppdrag/2024/SVK_2024/pythonkörningar\wires_250519.gdb into PostGIS as 'LG031_4_fas'.
Successfully imported C:/Uppdrag/2024/SVK_2024/pythonkörningar\wires_250519.gdb into PostGIS as 'LG031_5_fas'.
Successfully imported C:/Uppdrag/2024/SVK_2024/pythonkörningar\wires_250519.gdb into PostGIS as 'LG031_6_fas'.
Successfully imported C:/Uppdrag/2024/SVK_2024/pythonkörningar\wires_250519.gdb into PostGIS as 'LG043_1_fas'.
S